In [1]:
from dotenv import load_dotenv
import os

import pandas as pd

from langchain_community.chat_models import ChatLlamaCpp
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_together import ChatTogether
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
import re
import json
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

from pydantic import BaseModel, Field
from typing import List, Dict
from enum import Enum


In [2]:
load_dotenv()

True

In [3]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [4]:
LOCAL_MODEL_PATH = "/Users/gautamnaik/models/Nous-Hermes-2-Mistral-7B-DPO.Q8_0.gguf"
# LOCAL_MODEL_PATH = "/Users/gautamnaik/models/llama-3-8B-function-calling-Q8_0.gguf"
# LOCAL_MODEL_PATH = "/Users/gautamnaik/models/functionary/functionary-7b-v2.q8_0.gguf"

In [5]:

tools = []

def get_local_llm():
    return ChatLlamaCpp(
        model_path=LOCAL_MODEL_PATH,
        temperature=0,
        max_tokens=512,
        n_ctx=2048,
        n_batch=512,
        n_threads=8,
        verbose=False,
    )

def get_google_llm():
    return ChatGoogleGenerativeAI(
        temperature=0,
        max_output_tokens=512,
        model="gemini-2.0-flash",
        # model="gemini-2.5-flash-preview-05-20",
        #  model= "gemini-2.5-pro-preview-05-06",
        verbose=False,
    )

def get_groq_llm():
    return ChatGroq(
        model="qwen-qwq-32b",
        temperature=0,
    )

def get_chat_together_llm():
    return ChatTogether(
        api_key=os.getenv("TOGETHER_AI_API_KEY"),
        temperature=0.0,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
    )

def get_openai_llm():
    return ChatOpenAI(
        model="gpt-4.1",
        temperature=0.0,
        api_key=os.getenv("OPENAI_API_KEY"),
        verbose=False,
    )


In [6]:
class GAD7Item(str, Enum):
    nervous = "nervous"
    uncontrollable_worry = "uncontrollable_worry"
    worrying_too_much = "worrying_too_much"
    trouble_relaxing = "trouble_relaxing"
    restlessness = "restlessness"
    irritability = "irritability"
    fear_of_something_awful = "fear_of_something_awful"


class TherapySession(BaseModel):
    therapy_session_number: int
    estimated_gad7_scores: Dict[GAD7Item, int]
    total_gad7_score: int
    justification: str


class GAD7AgentOutput(BaseModel):
    client_id: str
    sessions: List[TherapySession]
    progress_summary: str


gad_7_scorer_agent = create_react_agent(
    model=get_chat_together_llm(),
    tools=[],
    prompt=(
        """
    You are a highly analytical psychological assessment agent specializing in anxiety scoring based on therapist notes.
    Your primary function is to estimate a GAD-7 anxiety score (Generalized Anxiety Disorder 7-item scale) for a client based on provided therapy session notes.
    You will then compare the scores from two or more consecutive sessions and report the change in anxiety levels. 

    **Instructions:**
    1.  **Analyze Each Therapy Session Individually:** Use the `therapy_session_number` as a reference to the therapy session number. For each therapy session provided, carefully read all the fields.
    2.  **Estimate GAD-7 Score per Session:** Map the described client's state and symptoms to the 7 GAD-7 items. Assign a score (0-3) for each item based on the *intensity*, *frequency*, and *duration* described in the notes.If an item is not explicitly mentioned but can be *reasonably inferred* from other descriptions (e.g., "overwhelmed" might infer "worrying too much"), make that inference and justify it. If no reasonable inference can be made, score 0.
    3.  **Calculate Total GAD-7 Score:** Sum the scores for all 7 items for each therapy session.
    4.  **Justify Scores:** Provide a concise justification for the total score of each session, referencing specific details (quotes if available, or paraphrased observations) from the notes that led to your score.
    5.  **Structured Output:** Return the scores and justifications in a structured JSON format for each therapy session and include the overall progress based on the score changes.

    **Output Format:**
    ```json
    {
    "client_id": "client1",
    "sessions": [
        {
        "therapy_session_number": 1,
        "estimated_gad7_scores": {
            "nervous": 2,
            "uncontrollable_worry": 2,
            "worrying_too_much": 2,
            "trouble_relaxing": 1,
            "restlessness": 0,
            "irritability": 1,
            "fear_of_something_awful": 1
        },
        "total_gad7_score": 9,
        "justification": "The client reported 'feeling anxious and stressed recently,' with symptoms described as moderate and lasting approximately six weeks. He felt 'overwhelmed' due to procrastination and taking on too many tasks, which supports moderate scores for items 1-3. Trouble relaxing is inferred from feeling overwhelmed (score 1). No evidence of restlessness (score 0). Some irritability is inferred from stress and overwhelm (score 1). Mild fear of something awful is inferred from the anxiety context (score 1). No mention of severe or persistent symptoms for higher scores."
        },
        {
        "therapy_session_number": 2,
        "estimated_gad7_scores": {
            "nervous": 1,
            "uncontrollable_worry": 1,
            "worrying_too_much": 1,
            "trouble_relaxing": 0,
            "restlessness": 0,
            "irritability": 0,
            "fear_of_something_awful": 0
        },
        "total_gad7_score": 3,
        "justification": "The client reported a 'significant reduction in anxiety and stress,' with symptoms now described as mild and occasional. He feels more confident and is managing tasks better, indicating improvement in items 1-3 (score 1 each). No evidence of trouble relaxing, restlessness, irritability, or fear, so these are scored 0. The overall tone is positive, and the client is actively using coping strategies."
        }
    ],
    "progress_summary": "The client's GAD-7 score decreased from 9 (moderate anxiety) in session 1 to 3 (minimal anxiety) in session 2, indicating significant improvement in anxiety symptoms and effective response to therapy interventions."
    }
    ```
    """
    ),
    name="gad_7_scorer_agent",
    response_format=GAD7AgentOutput.model_json_schema()
)

# for chunk in gad_7_scorer_agent.stream(
#     {"messages": [{"role": "user", "content": """ 
# ```json\n
#     '[{"therapy_session_number": 1, "client_id": "client1", "chief_complaint": "The client reported feeling anxious and stressed recently, which is unusual for him. He also mentioned a tendency to take on tasks and procrastinate, leading to feelings of being overwhelmed.", "symptoms": [{"description": "Anxiety and stress", "onset": "Approximately six weeks ago", "frequency": "Intermittent, with episodes lasting a couple of weeks", "ascendance": "The symptoms have leveled off after the initial episodes", "intensity": "Moderate", "duration": "Approximately six weeks", "quote": "\\"I probably, probably a couple of, maybe six weeks ago, something. Yeah, I see that as quite recent.\\""}], "diagnosis": [{"dsm_5_diagnosis": "Generalized Anxiety Disorder", "dsm_5_code": "300.02", "icd_10_code": "F41.1"}], "mood_and_affect": "The client\'s mood was anxious, and his affect was congruent with his reported experiences.", "speech_and_language": "The client\'s speech was coherent, structured, and pertinent to the context.", "thought_process_and_content": "The client\'s thought process was organized, and his content was relevant to the discussion.", "orientation": "The client was oriented to person, place, time, and situation.", "perceptual_disturbances": "No perceptual disturbances were reported or observed.", "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.", "insight": "The client demonstrated a good level of insight into his symptoms and behaviors, recognizing the impact of his anxiety and stress on his functioning.", "risks_or_safety_concerns": "No Indication of Risk", "hopelessness": "No hopelessness expressed or observed.", "suicidal_thoughts_or_attempts": "No suicidal thoughts or attempts reported or suspected.", "self_harm": "No self-harm reported or suspected.", "dangerous_to_others": "No danger to others reported or suspected.", "risk_quote": "NA", "response_to_treatment": "NA"}, {"therapy_session_number": 2, "client_id": "client1", "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.", "symptoms": [{"description": "Anxiety and stress", "onset": "Six weeks ago, significantly reduced in the past two weeks", "frequency": "Occasional, with noticeable improvement", "ascendance": "Symptoms have decreased following the application of therapy strategies", "intensity": "Mild", "duration": "Minimal symptoms over the past two weeks", "quote": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""}], "diagnosis": [{"dsm_5_diagnosis": "Generalized Anxiety Disorder", "dsm_5_code": "300.02", "icd_10_code": "F41.1"}], "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.", "speech_and_language": "The client\'s speech was coherent, upbeat, and reflective of his progress.", "thought_process_and_content": "The client\'s thought process was organized, and his content was focused on the positive changes he has made.", "orientation": "The client was oriented to person, place, time, and situation.", "perceptual_disturbances": "No perceptual disturbances were reported or observed.", "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.", "insight": "The client demonstrated a strong level of insight into his progress, recognizing the positive impact of the strategies he has implemented.", "risks_or_safety_concerns": "No Indication of Risk", "hopelessness": "No hopelessness expressed or observed.", "suicidal_thoughts_or_attempts": "No suicidal thoughts or attempts reported or suspected.", "self_harm": "No self-harm reported or suspected.", "dangerous_to_others": "No danger to others reported or suspected.", "risk_quote": "NA", "response_to_treatment": "The client has shown significant improvement, actively participating in therapy and implementing the strategies discussed."}]'
# """}]}
# ):
#     pretty_print_messages(chunk)

In [7]:
res =gad_7_scorer_agent.invoke(
    {"messages": [{"role": "user", "content": """ 
```json\n
'[\n    {\n        "therapy_session_number": 1,\n        "client_id": "client1",\n        "chief_complaint": "The client reported feeling anxious and stressed recently, which is unusual for him. He also mentioned a tendency to take on tasks and procrastinate, leading to feelings of being overwhelmed.",\n        "symptoms": [\n            {\n                "description": "Anxiety and stress",\n                "onset": "Approximately six weeks ago",\n                "frequency": "Intermittent, with episodes lasting a couple of weeks",\n                "ascendance": "The symptoms have leveled off after the initial episodes",\n                "intensity": "Moderate",\n                "duration": "Approximately six weeks",\n                "quote": "\\"I probably, probably a couple of, maybe six weeks ago, something. Yeah, I see that as quite recent.\\""\n            }\n        ],\n        "sleep": "NA",\n        "physical_activity": "NA",\n        "mood_and_affect": "The client\'s mood was anxious, and his affect was congruent with his reported experiences.",\n        "thought_process_and_content": "The client\'s thought process was organized, and his content was relevant to the discussion.",\n        "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.",\n        "speech_and_language": "The client\'s speech was coherent, structured, and pertinent to the context.",\n        "risk_quote": "NA",\n        "diagnosis": [\n            {\n                "dsm_5_diagnosis": "Generalized Anxiety Disorder",\n                "dsm_5_code": "300.02",\n                "icd_10_code": "F41.1"\n            }\n        ]\n    },\n    {\n        "therapy_session_number": 2,\n        "client_id": "client1",\n        "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.",\n        "symptoms": [\n            {\n                "description": "Anxiety and stress",\n                "onset": "Six weeks ago, significantly reduced in the past two weeks",\n                "frequency": "Occasional, with noticeable improvement",\n                "ascendance": "Symptoms have decreased following the application of therapy strategies",\n                "intensity": "Mild",\n                "duration": "Minimal symptoms over the past two weeks",\n                "quote": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n            }\n        ],\n        "sleep": "Improved sleep patterns were reported, with more restful nights.",\n        "physical_activity": "The client has increased his physical activity, incorporating regular walks into his routine.",\n        "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.",\n        "thought_process_and_content": "The client\'s thought process was organized, and his content was focused on the positive changes he has made.",\n        "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.",\n        "speech_and_language": "The client\'s speech was coherent, upbeat, and reflective of his progress.",\n        "risk_quote": "NA",\n        "diagnosis": [\n            {\n                "dsm_5_diagnosis": "Generalized Anxiety Disorder",\n                "dsm_5_code": "300.02",\n                "icd_10_code": "F41.1"\n            }\n        ]\n    }\n]'"""}]}
)

In [8]:
res

{'messages': [HumanMessage(content=' \n```json\n\n\'[\n    {\n        "therapy_session_number": 1,\n        "client_id": "client1",\n        "chief_complaint": "The client reported feeling anxious and stressed recently, which is unusual for him. He also mentioned a tendency to take on tasks and procrastinate, leading to feelings of being overwhelmed.",\n        "symptoms": [\n            {\n                "description": "Anxiety and stress",\n                "onset": "Approximately six weeks ago",\n                "frequency": "Intermittent, with episodes lasting a couple of weeks",\n                "ascendance": "The symptoms have leveled off after the initial episodes",\n                "intensity": "Moderate",\n                "duration": "Approximately six weeks",\n                "quote": "\\"I probably, probably a couple of, maybe six weeks ago, something. Yeah, I see that as quite recent.\\""\n            }\n        ],\n        "sleep": "NA",\n        "physical_activity": "NA",

In [11]:
res["structured_response"]

{'client_id': 'client1',
 'progress_summary': "The client's GAD-7 score decreased from 9 (moderate anxiety) in session 1 to 3 (minimal anxiety) in session 2, indicating significant improvement in anxiety symptoms and effective response to therapy interventions.",
 'sessions': [{'estimated_gad7_scores': {'fear_of_something_awful': 1,
    'irritability': 1,
    'nervous': 2,
    'restlessness': 0,
    'trouble_relaxing': 1,
    'uncontrollable_worry': 2,
    'worrying_too_much': 2},
   'justification': "The client reported feeling anxious and stressed, which is unusual for him, with symptoms described as moderate and lasting approximately six weeks. He felt overwhelmed due to procrastination and taking on too many tasks, which supports moderate scores for items 1-3 (nervous, uncontrollable worry, worrying too much). Trouble relaxing is inferred from feeling overwhelmed (score 1). No evidence of restlessness (score 0). Some irritability is inferred from stress and overwhelm (score 1). Mil

In [12]:
class PHQ9Item(str, Enum):
    little_interest = "little_interest"
    feeling_down = "feeling_down"
    trouble_sleeping = "trouble_sleeping"
    feeling_tired = "feeling_tired"
    poor_appetite = "poor_appetite"
    feeling_bad_about_self = "feeling_bad_about_self"
    trouble_concentrating = "trouble_concentrating"
    slow_or_fast = "slow_or_fast"
    thoughts_of_self_harm = "thoughts_of_self_harm"


class TherapySessionPHQ9(BaseModel):
    therapy_session_number: int
    estimated_phq9_scores: Dict[PHQ9Item, int]
    total_phq9_score: int
    justification: str


class PHQ9AgentOutput(BaseModel):
    client_id: str
    sessions: List[TherapySessionPHQ9]
    progress_summary: str


phq_9_scorer_agent = create_react_agent(
    model=get_chat_together_llm(),
    tools=[],
    prompt=(
        """
    You are a highly analytical psychological assessment agent specializing in depression scoring based on therapist notes.
    Your primary function is to estimate a PHQ-9 depression score (Patient Health Questionnaire-9) for a client based on provided therapy session notes.
    You will then compare the scores from two or more consecutive sessions and report the change in depression levels.

    **Instructions:**
    1.  **Analyze Each Therapy Session Individually:** Use the `therapy_session_number` as a reference to the therapy session number. For each therapy session provided, carefully read all the fields.
    2.  **Estimate PHQ-9 Score per Session:** Map the described client's state and symptoms to the 9 PHQ-9 items. Assign a score (0-3) for each item based on the *intensity*, *frequency*, and *duration* described in the notes.
        -   **0 = Not at all**
        -   **1 = Several days**
        -   **2 = More than half the days**
        -   **3 = Nearly every day**
        If an item is not explicitly mentioned but can be *reasonably inferred* from other descriptions (e.g., "lack of motivation" might infer "little interest or pleasure in doing things"), make that inference and justify it. If no reasonable inference can be made, score 0.
    3.  **Calculate Total PHQ-9 Score:** Sum the scores for all 9 items for each therapy session.
    4.  **Justify Scores:** Provide a concise justification for the total score of each session, referencing specific details (quotes if available, or paraphrased observations) from the notes that led to your score.
    5.  **Structured Output:** Return the scores and justifications in a structured JSON format for each therapy session and include the overall progress based on the score changes.

    **Output Format:**
    ```json
    {
    "client_id": "client1",
    "sessions": [
        {
        "therapy_session_number": 1,
        "estimated_phq9_scores": {
            "little_interest": 2,
            "feeling_down": 3,
            "trouble_sleeping": 2,
            "feeling_tired": 2,
            "poor_appetite": 1,
            "feeling_bad_about_self": 2,
            "trouble_concentrating": 1,
            "slow_or_fast": 0,
            "thoughts_of_self_harm": 0
        },
        "total_phq9_score": 13,
        "justification": "The client reported feeling 'constantly sad and lacking motivation for the past few weeks,' which supports high scores for feeling down (3) and little interest (2). She mentioned 'difficulty falling asleep most nights' (2) and feeling 'drained of energy' (2). Appetite was 'somewhat reduced' (1). She expressed 'guilt about not doing enough' (2). Concentration was 'a bit off' (1). No explicit mention of psychomotor changes or self-harm thoughts, so scored 0. Overall symptoms suggest moderately severe depression."
        },
        {
        "therapy_session_number": 2,
        "estimated_phq9_scores": {
            "little_interest": 1,
            "feeling_down": 1,
            "trouble_sleeping": 1,
            "feeling_tired": 1,
            "poor_appetite": 0,
            "feeling_bad_about_self": 0,
            "trouble_concentrating": 0,
            "slow_or_fast": 0,
            "thoughts_of_self_harm": 0
        },
        "total_phq9_score": 4,
        "justification": "The client reported feeling 'better and more engaged in activities,' with 'some lingering sadness on a few days,' supporting mild scores for little interest (1) and feeling down (1). Sleep has 'improved but still occasionally restless' (1), and energy levels are 'better but still not fully restored' (1). No mention of appetite issues, self-worth issues, or concentration problems, so scored 0 for these. No indication of psychomotor changes or self-harm thoughts. Overall symptoms suggest minimal depression, indicating good progress."
        }
    ],
    "progress_summary": "The client's PHQ-9 score decreased from 13 (moderately severe depression) in session 1 to 4 (minimal depression) in session 2, indicating significant improvement in depressive symptoms and positive response to therapeutic interventions."
    }
    ```
    """
    ),
    name="phq_9_scorer_agent",
    response_format=PHQ9AgentOutput.model_json_schema()
)

In [13]:
res =phq_9_scorer_agent.invoke(
    {"messages": [{"role": "user", "content": """ 
```json\n
                   [\n    {\n        "therapy_session_number": 1,\n        "client_id": "client3",\n        "chief_complaint": "The client is experiencing persistent low mood, lack of motivation, and feelings of hopelessness since graduating college.",\n        "symptoms": [\n            {\n                "description": "Persistent low mood and sadness.",\n                "onset": "Approximately 3 months ago, coinciding with college graduation.",\n                "frequency": "Daily",\n                "ascendance": "Persistent at current baseline.",\n                "intensity": "Moderate",\n                "duration": "3 months",\n                "quote": "\\"It\'s like a cloud is always over me.\\""\n            },\n            {\n                "description": "Lack of motivation and anhedonia (loss of interest or pleasure).",\n                "onset": "Approximately 3 months ago.",\n                "frequency": "Daily",\n                "ascendance": "Persistent at current baseline.",\n                "intensity": "Moderate",\n                "duration": "3 months",\n                "quote": "\\"Nothing I used to enjoy feels fun anymore.\\""\n            }\n        ],\n        "sleep": "The client reports increased sleep (hypersomnia), often sleeping 10-12 hours per day.",\n        "nutrition": "The client reports a decreased appetite and often skips meals.",\n        "mood_and_affect": "The client reported a sad mood and displayed a restricted and congruent affect.",\n        "cognition": "The client\'s cognitive functioning appeared intact, though they reported some difficulty concentrating.",\n        "hopelessness": "The client expressed feelings of hopelessness about their future and ability to find a job.",\n        "suicidal_thoughts_or_attempts": "No active suicidal ideation but some passive thoughts of not wanting to wake up.",\n        "self_harm": "No indication of self-harm.",\n        "diagnosis": [\n            {\n                "dsm_5_diagnosis": "Major Depressive Disorder, Single Episode, Moderate",\n                "dsm_5_code": "F32.1",\n                "icd_10_code": "F32.1"\n            }\n        ]\n    },\n    {\n        "therapy_session_number": 2,\n        "client_id": "client3",\n        "chief_complaint": "The client continues to experience significant low mood, anhedonia, and a profound lack of motivation, which has worsened over time.",\n        "symptoms": [\n            {\n                "description": "Increased low mood and persistent sadness.",\n                "onset": "Symptoms have persisted and intensified over the past 3 months.",\n                "frequency": "All day, every day.",\n                "ascendance": "Symptoms are worsening, with the client feeling increasingly numb.",\n                "intensity": "High",\n                "duration": "3 months",\n                "quote": "\\"It feels like I\'m just watching life happen around me.\\""\n            },\n            {\n                "description": "Profound lack of motivation and anhedonia.",\n                "onset": "Symptoms have persisted and intensified over the past 3 months.",\n                "frequency": "All day, every day.",\n                "ascendance": "Symptoms are worsening, with the client feeling unable to initiate any activity.",\n                "intensity": "High",\n                "duration": "3 months",\n                "quote": "\\"I know I should do things, but I just can\'t bring myself to start.\\""\n            }\n        ],\n        "sleep": "The client reports increased sleep (hypersomnia) and difficulty getting out of bed.",\n        "nutrition": "The client\'s appetite remains decreased, and they report eating very little.",\n        "mood_and_affect": "The client appeared profoundly sad and expressed feelings of overwhelming emptiness. Their affect was flat and non-reactive.",\n        "cognition": "The client\'s cognitive functioning appeared intact but severely slowed under the weight of depression.",\n        "hopelessness": "The client expressed profound feelings of hopelessness about their future.",\n        "suicidal_thoughts_or_attempts": "Passive suicidal ideation (e.g., \\"I just don\'t want to exist anymore\\") but no active plan or intent.",\n        "self_harm": "No indication of self-harm.",\n        "diagnosis": [\n            {\n                "dsm_5_diagnosis": "Major Depressive Disorder, Single Episode, Severe",\n                "dsm_5_code": "F32.2",\n                "icd_10_code": "F32.2"\n            }\n        ]\n    }\n]
                   ```
                   """}]
}
)

In [15]:
res["structured_response"]

{'client_id': 'client3',
 'progress_summary': "The client's PHQ-9 score increased from 16 (severe depression) in session 1 to 22 (very severe depression) in session 2, indicating a significant worsening of depressive symptoms. This suggests that the client's condition has deteriorated over time, and therapeutic interventions may need to be adjusted to address the increasing severity of symptoms.",
 'sessions': [{'estimated_phq9_scores': {'feeling_bad_about_self': 2,
    'feeling_down': 3,
    'feeling_tired': 2,
    'little_interest': 3,
    'poor_appetite': 2,
    'slow_or_fast': 0,
    'thoughts_of_self_harm': 1,
    'trouble_concentrating': 1,
    'trouble_sleeping': 2},
   'justification': "The client's persistent low mood, lack of motivation, and feelings of hopelessness support high scores for feeling down and little interest. The increased sleep, decreased appetite, and difficulty concentrating also contribute to the overall score. The presence of passive suicidal thoughts, thou